In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from catboost import Pool
import catboost as cat
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e11/sample_submission.csv
/kaggle/input/playground-series-s4e11/train.csv
/kaggle/input/playground-series-s4e11/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s4e11/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e11/test.csv')

In [3]:
train.head(5)

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


In [4]:
train.nunique().to_frame()

,0
id,140700
Name,422
Gender,2
Age,43
City,98
Working Professional or Student,2
Profession,64
Academic Pressure,5
Work Pressure,5
CGPA,331


In [5]:
categorical_cols = [col for col in train.columns if col!='Depression']

In [6]:
train[categorical_cols] = train[categorical_cols].astype('str').astype('category')
X = train[categorical_cols]
y = train['Depression']

In [7]:
X.dtypes

id                                       category
Name                                     category
Gender                                   category
Age                                      category
City                                     category
Working Professional or Student          category
Profession                               category
Academic Pressure                        category
Work Pressure                            category
CGPA                                     category
Study Satisfaction                       category
Job Satisfaction                         category
Sleep Duration                           category
Dietary Habits                           category
Degree                                   category
Have you ever had suicidal thoughts ?    category
Work/Study Hours                         category
Financial Stress                         category
Family History of Mental Illness         category
dtype: object

In [8]:
train_pool = Pool(data=X, label=y, cat_features=X[categorical_cols].columns.values)

In [9]:
cb = cat.CatBoostClassifier(random_state=42,iterations=4000, l2_leaf_reg= 0.7, random_strength=0.7, learning_rate=0.015558329, verbose=0, objective='Logloss', eval_metric='AUC')

In [10]:
cb.fit(train_pool)

In [11]:
test[categorical_cols] = test[categorical_cols].astype('str').astype('category')

In [12]:
test_pool = Pool(data=test, cat_features=test[categorical_cols].columns.values)
preds = cb.predict(test_pool)

In [13]:
submission_df = pd.DataFrame({
    'id':test['id'],
    'Depression':preds
})
submission_df.to_csv('submission.csv', index=False)